<a href="https://colab.research.google.com/github/steeleb/lakeCoSTR_colab/blob/dev/lakeCoSTRv1_12.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **lakeCoSTR for Colab**<br>
a tool for *__lake__ __Co__ llection 2 __S__ urface __T__ emperature __R__ etrieval*<p>
<p>

**Version 1.12**<br>
*Last updated: 2022-09-06<p>*

Created by:<br>
Christina Herrick<br>
University of New Hampshire<br>
christina.herrick@unh.edu<p>
Bethel Steele<br>
Cary Institute of Ecosystem Studies<br>
steeleb@caryinstitute.org<p>
__This tool is still under development and has an associated manuscript that has been submitted for peer review. This tool is licensed under CC-BY-NC-SA, please cite accordingly.__

# **User Guide**
Please review the [**user guide**](https://github.com/lakeCoSTR/lakeCoSTR_colab/blob/main/UserGuide_lakeCoSTR_colab.md) for assistance in navigating the tool and for specific constraints and caveats.

# **Motivations of lakeCoSTR**
We created lakeCoSTR to make the acquisition and analysis of Landsat's Collection 2 Surface Temperature product more accessible for those interested in obtaining historical estimates of lake temperature from remote sensing data. This tool is geared to researchers interested in gathering historical temperature estimates, especially for small lakes.

# **1. Initial Setup**


## **1.1 Modules**

This section of code blocks imports necessary python modules for the notebook to run. You will be prompted to click one or more URLs and be taken to a page to sign in with your Google account.

Copy the unique code(s) provided on the web page when prompted and paste where prompted to finish authorization.

In [ ]:
#@markdown __Run this block__ to authorize Colab to authenticate your Google account and give it access to upload files from your local computer.
from google.colab import auth, files
auth.authenticate_user()
import gspread
from oauth2client.client import GoogleCredentials

In [ ]:
#@markdown __Run this block__ to connect Colab to your Earth Engine account
#@markdown You must already be authorized to use Google Earth Engine. 
#@markdown <p>If you do not already have access, <a href="https://signup.earthengine.google.com/#!/" target="_blank">fill out this application</a>.
import ee
ee.Authenticate()
ee.Initialize()

To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://code.earthengine.google.com/client-auth?scopes=https%3A//www.googleapis.com/auth/earthengine%20https%3A//www.googleapis.com/auth/devstorage.full_control&request_id=ArLCxADopvmXcmgTSLU2k2NlHvKn-5q6qYZQXohwaXY&tc=lcorlq8VUktxIzR69nGqbsWkJcdncEbSIK1aQB_XOk4&cc=cN0HOJaepG0zYDG7_2WKmU9iNGL1x65-AsrpvUSPBB8

The authorization workflow will generate a code, which you should paste in the box below.
Enter verification code: 4/1AdQt8qj90ZaSL0sK5_D2nk2DUq7c3nEkBcgU8XlhpyvRD90xQW4Pkm--6YU

Successfully saved authorization token.


In [ ]:
#@markdown __Run this block__ to connect Colab to your Google Drive
from google.colab import drive
drive.mount('/content/drive', force_remount=False)

Mounted at /content/drive


In [ ]:
#@markdown __Run this block__ to install packages and functions used for 
#@markdown interactive mapping and data exploration
import pandas as pd
import numpy as np
import os
import re
import subprocess
import matplotlib.pyplot as plt
from time import strftime, sleep
from datetime import datetime, timedelta
import pytz
import folium
from folium import plugins
from google.colab import data_table

try:
  from ipyleaflet import Map, DrawControl
  from ipywidgets import Layout
except ImportError:
  subprocess.check_call(["python","-m","pip","install","-U","ipyleaflet"])
  from ipyleaflet import Map, basemaps, basemap_to_tiles, DrawControl
  from ipywidgets import Layout

# Add custom basemaps to folium
basemaps = {
    'Google Maps': folium.TileLayer(
        tiles = 'https://mt1.google.com/vt/lyrs=m&x={x}&y={y}&z={z}',
        attr = 'Google',
        name = 'Google Maps',
        overlay = True,
        control = True
    ),
    'Google Satellite': folium.TileLayer(
        tiles = 'https://mt1.google.com/vt/lyrs=s&x={x}&y={y}&z={z}',
        attr = 'Google',
        name = 'Google Satellite',
        overlay = True,
        control = True
    ),
    'Google Terrain': folium.TileLayer(
        tiles = 'https://mt1.google.com/vt/lyrs=p&x={x}&y={y}&z={z}',
        attr = 'Google',
        name = 'Google Terrain',
        overlay = True,
        control = True
    ),
    'Google Satellite Hybrid': folium.TileLayer(
        tiles = 'https://mt1.google.com/vt/lyrs=y&x={x}&y={y}&z={z}',
        attr = 'Google',
        name = 'Google Satellite',
        overlay = True,
        control = True
    ),
    'Esri Satellite': folium.TileLayer(
        tiles = 'https://server.arcgisonline.com/ArcGIS/rest/services/World_Imagery/MapServer/tile/{z}/{y}/{x}',
        attr = 'Esri',
        name = 'Esri Satellite',
        overlay = True,
        control = True
    )
}

# Define a method for displaying Earth Engine image tiles on a folium map.
def add_ee_layer(self, ee_object, vis_params, name):
    
    try:    
        # display ee.Image()
        if isinstance(ee_object, ee.image.Image):    
            map_id_dict = ee.Image(ee_object).getMapId(vis_params)
            folium.raster_layers.TileLayer(
            tiles = map_id_dict['tile_fetcher'].url_format,
            attr = 'Google Earth Engine',
            name = name,
            overlay = True,
            control = True
            ).add_to(self)
        # display ee.ImageCollection()
        elif isinstance(ee_object, ee.imagecollection.ImageCollection):    
            ee_object_new = ee_object.mosaic()
            map_id_dict = ee.Image(ee_object_new).getMapId(vis_params)
            folium.raster_layers.TileLayer(
            tiles = map_id_dict['tile_fetcher'].url_format,
            attr = 'Google Earth Engine',
            name = name,
            overlay = True,
            control = True
            ).add_to(self)
        # display ee.Geometry()
        elif isinstance(ee_object, ee.geometry.Geometry):    
            folium.GeoJson(
            data = ee_object.getInfo(),
            name = name,
            overlay = True,
            control = True
        ).add_to(self)
        # display ee.FeatureCollection()
        elif isinstance(ee_object, ee.featurecollection.FeatureCollection):  
            ee_object_new = ee.Image().paint(ee_object, 0, 2)
            map_id_dict = ee.Image(ee_object_new).getMapId(vis_params)
            folium.raster_layers.TileLayer(
            tiles = map_id_dict['tile_fetcher'].url_format,
            attr = 'Google Earth Engine',
            name = name,
            overlay = True,
            control = True
        ).add_to(self)
    
    except:
        print("Could not display {}".format(name))
    
# Add EE drawing method to folium.
folium.Map.add_ee_layer = add_ee_layer

print("Packages installed")

Packages installed


## **1.2 Imported variables**
These are Landsat-specific variable settings for GEE.

In [ ]:
#@markdown __Run this block__ to import pre-existing features, images, and collections from Earth Engine
wrs2 = ee.FeatureCollection("users/christinaherrickunh/WRS2_descending_2018")
utmbounds = ee.FeatureCollection("users/christinaherrickunh/UTM_Zone_Boundaries")
sw = ee.Image("JRC/GSW1_2/GlobalSurfaceWater")
openwater = sw.select("occurrence")
l4t1 = ee.ImageCollection("LANDSAT/LT04/C02/T1_L2")
l4t2 = ee.ImageCollection("LANDSAT/LT04/C02/T2_L2")
l5t1 = ee.ImageCollection("LANDSAT/LT05/C02/T1_L2")
l5t2 = ee.ImageCollection("LANDSAT/LT05/C02/T2_L2")
l7t1 = ee.ImageCollection("LANDSAT/LE07/C02/T1_L2")
l7t2 = ee.ImageCollection("LANDSAT/LE07/C02/T2_L2")
l8t1 = ee.ImageCollection("LANDSAT/LC08/C02/T1_L2")
l8t2 = ee.ImageCollection("LANDSAT/LC08/C02/T2_L2")


def prep457bands(img):
  systime = img.get('system:time_start')
  elev = img.get("SUN_ELEVATION")
  sza = ee.Number(90).subtract(elev)
  
  qa = img.select(["QA_PIXEL"])
  cloud1 = qa.bitwiseAnd(2).eq(0)  # bit 1, dilated cloud
  cloud3 = qa.bitwiseAnd(8).eq(0)  # bit 3, cloud
  cloudshadow = qa.bitwiseAnd(16).eq(0) # bit 4, cloud shadow
  snow = qa.bitwiseAnd(32).eq(0)  # bit 5, snow
  cloud_confid = qa.rightShift(8).bitwiseAnd(3).lt(2)  # bits 8-9
  cloudsh_confid = qa.rightShift(10).bitwiseAnd(3).lt(2)  # bits 10-11
  snow_confid = qa.rightShift(12).bitwiseAnd(3).lt(2)  # bits 12-13
  cirrus_confid = qa.rightShift(14).bitwiseAnd(3).lt(2)  # bits 14-15

  temp = img.select(["ST_B6"],["surface_temp"]).multiply(0.00341802).add(149).add(-273.15)
  
  updated = (temp.updateMask(cloud1)
  .updateMask(cloud3)
  .updateMask(snow)
  .updateMask(cloudshadow)
  .updateMask(cloud_confid)
  .updateMask(snow_confid)
  .updateMask(cirrus_confid)
  .updateMask(cloudsh_confid))
          
  return ee.Image(updated).copyProperties(img).set("system:time_start",systime).set("SOLAR_ZENITH_ANGLE", sza)

def prep8bands(img):
  systime = img.get('system:time_start')
  elev = img.get("SUN_ELEVATION")
  sza = ee.Number(90).subtract(elev)

  qa = img.select(["QA_PIXEL"])
  cloud1 = qa.bitwiseAnd(2).eq(0)  # bit 1, dilated cloud
  cloud2 = qa.bitwiseAnd(4).eq(0)  # bit 2, cirrus (high confidence)
  cloud3 = qa.bitwiseAnd(8).eq(0)  # bit 3, cloud
  cloudshadow = qa.bitwiseAnd(16).eq(0) # bit 4, cloud shadow
  snow = qa.bitwiseAnd(32).eq(0)  # bit 5, snow
  cloud_confid = qa.rightShift(8).bitwiseAnd(3).lt(2)  # bits 8-9
  cloudsh_confid = qa.rightShift(10).bitwiseAnd(3).lt(2)  # bits 10-11
  snow_confid = qa.rightShift(12).bitwiseAnd(3).lt(2)  # bits 12-13
  cirrus_confid = qa.rightShift(14).bitwiseAnd(3).lt(2)  # bits 14-15
  
  temp = img.select(["ST_B10"],["surface_temp"]).multiply(0.00341802).add(149).add(-273.15)

  updated = (temp.updateMask(cloud1)
    .updateMask(cloud2)
    .updateMask(cloud3)
    .updateMask(cloudshadow)
    .updateMask(snow)
    .updateMask(cloud_confid)
    .updateMask(cloudsh_confid)
    .updateMask(snow_confid)
    .updateMask(cirrus_confid))

  return ee.Image(updated).copyProperties(img).set("system:time_start",systime).set("SOLAR_ZENITH_ANGLE", sza)


print("variables and functions imported")

variables and functions imported


## **1.3 Define the lake area**

In this section, use either *Option A*, where you define the bounding box, or *Option B* where you import a shape file of your lake. 


###Option A: use a bounding box
Use the Rectangle tool on the map to draw a bounding box. The bounding box should be as small as possible while still including the entire lake surface. It cannot be a centroid location.<p>Do not run this block if you are using Option B.

In [ ]:
#@markdown Run this block to generate a map and use the Rectangle tool to draw a bounding box. 
#@markdown The bounding box should be as small as possible while still including the entire lake surface. It cannot be a centroid location.
#@markdown If more than one box is drawn, the most recently drawn box will be used.
#@markdown <p>Do not run this block if you are using Option B.

from ipyleaflet import SearchControl, Marker, AwesomeIcon

'''pure leaflet'''
# baseMap = basemap_to_tiles(basemaps.CartoDB.DarkMatter)
m = Map(center=(15,0), zoom=2, scroll_wheel_zoom=True, layout=Layout(width='80%', height='500px'))

draw_control = DrawControl(rectangle={"shapeOptions":{"fillColor":"#3fca45d", "color":"#efed69", "fillOpacity": 0.25}}, circle={}, circlemarker={}, polygon={}, polyline={})
search_control = SearchControl(position="topright", url='https://nominatim.openstreetmap.org/search?format=json&q={s}', zoom=7)

gj = []
def handle_draw(self, action, geo_json):
  if action in ['created','edited']:
    gj.append(geo_json)
    box = ee.Geometry.Polygon(gj[-1]["geometry"]["coordinates"],None, False, 100)
    rec_geojson = gj[-1]["geometry"]["coordinates"]
    lon1, lat1 = rec_geojson[0][0]
    lon2, lat2 = rec_geojson[0][2]

draw_control.on_draw(handle_draw)
m.add_control(draw_control)
m.add_control(search_control)
m


Map(center=[15, 0], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom_out_text…

In [ ]:
#@markdown Run this block to set up your bounding box coordinates.
use_user_input_file = "no"
try:
  box = ee.Geometry.Polygon(gj[-1]["geometry"]["coordinates"],None, False, 100)
  rec_geojson = gj[-1]["geometry"]["coordinates"]
  lon1, lat1 = rec_geojson[0][0]
  lon2, lat2 = rec_geojson[0][2]

  center_lat = (lat1+lat2)/2
  center_lon = (lon1+lon2)/2

  print(f'''
  north bounds: {lat1}
  south bounds: {lat2}
  west bounds: {lon1}
  east bounds: {lon2}
  ''')
except IndexError:
  print("An error occurred, usually because this block was run but no box was drawn on the above map")


  north bounds: 43.317185
  south bounds: 43.434972
  west bounds: -72.089878
  east bounds: -72.018122
  


###Option B: upload a table file
Using your Google Earth Engine account, upload a shapefile or csv of your study lake to your Assets and link to it here.<p>Click [here](https://drive.google.com/file/d/1lfFoZzQD_wAA7Bnil7mI4zwjKDG5kegh/view?usp=sharing) and [here](https://drive.google.com/file/d/1KNaZV63J_LUp6X1OcoLX1wonF0ASiIzi/view?usp=sharing) for screenshots of the process. 

In [ ]:
#@markdown Copy the asset path from Google Earth Engine and change
#@markdown dropdown to `yes`.
#@markdown File path should begin with 'users/'
user_input_file = "" #@param {type: "string"}
use_user_input_file = "no" #@param ["yes","no"] {allow-input: false}
#@markdown Remember to __run this block__ after filling in the above fields.
#@markdown Do not run this block if you are using Option A.

if use_user_input_file=="no":
  print("If you intend to use Option B, change the 'use_user_input_file' variable to 'yes' and rerun")
elif len(user_input_file)>1:
  shp = ee.FeatureCollection(user_input_file).geometry()
  centroid = shp.centroid(10).getInfo()["coordinates"]
  center_lat = centroid[1]
  center_lon = centroid[0]
  print(center_lat)
  print(center_lon)
else:
  print("No file input; will use Opt A bounding box coordinates")
  use_user_input_file = "no"

if 'shp' in globals() or 'shp' in locals():
  shp_map = folium.Map(location=[center_lat,center_lon],zoom_start=10,width=600,height=600)
  shp_map.add_child(folium.LatLngPopup())
  basemaps['Google Maps'].add_to(shp_map)
  shp_map.add_ee_layer(shp,{},'aoi')
  display(shp_map)
# else:
#   print("Display does not apply here; Opt A bounding box is being used for AOI")

## **1.4 Set remaining user variables**

This section defines acceptable water persistence and error for the Landsat pixels, the time period of interest, and confirms the path and row for Landsat acquisition. You can also change the default Google Drive folder for any outputs.

In [ ]:
#@markdown We suggest saving the lakeCoSTR data to it's own folder in your Google Drive. 
#@markdown Replace your lake name where the following prompts read 'LAKENAME'. These folders
#@markdown will be automatically created within this script.

lake_name = 'sunapee_v1_12' #@param {type:"string"}
output_dir = "/content/drive/MyDrive/lakeCoSTR_output/"+ lake_name

#this parses the 'output_dir' string to get last listed subfolder
short_dir = lake_name
file_prefix = lake_name
print(f"Your data will be saved in {output_dir}")

if not os.path.exists(output_dir):
  os.makedirs(output_dir)

def check_splcharacter(test):
  string_check = re.compile('[@!#$%^&*() <>?/\|}{~:]')
  if not string_check.search(test)==None:
    print("Your file prefix contains special characters, please fix")
    lake_name = "LAKENAME"
  else:
    print("Files will all have the prefix:",test)

check_splcharacter(file_prefix)

Your data will be saved in /content/drive/MyDrive/lakeCoSTR_output/sunapee_v1_12
Files will all have the prefix: sunapee_v1_12


In [ ]:
#@markdown Boundaries of water bodies are automatically detected using the JRC 
#@markdown Global Surface Water Mapping Layer dataset of percent water occurrence. 
#@markdown By default, a pixel has to be classified as water at least 55% of the 
#@markdown time. This is defined as `pctTime`. If your lake of interest experiences 
#@markdown significant fluctuations of lake level, we suggest using a higher percentage than the default. 

#@markdown <p>More information on this dataset can be found at https://doi.org/10.1038/nature20584



pctTime = 55  #@param {type: "number"}
#@markdown __Run this block__ after inputting the above parameter

water_fig = folium.Figure(width="60%")
water_map = folium.Map(location=[center_lat,center_lon],zoom_start=10,tiles=None).add_to(water_fig)
water_map.add_child(folium.LatLngPopup())

# blues = ["f7fbff","deebf7","c6dbef","9ecae1","6baed6","4292c6","2171b5","08519c","08306b"]
blues = ["FFFFFF","08306b"]

# basemaps['Google Satellite'].add_to(water_map)
folium.TileLayer('cartodbdark_matter').add_to(water_map)
water_map.add_ee_layer(box,{},'aoi')
water_map.add_ee_layer(openwater,{"min":pctTime, "max":100, "palette":blues}, 'water presence')
# water_map.add_ee_layer(box,{},'aoi')
print("Occurrence of water detected by Landsat where water is present at least ",pctTime,"% of the time (in blue)")
print("Water occurrence less than ", pctTime, "% of the time are in white")
display(water_fig)

Occurrence of water detected by Landsat where water is present at least  55 % of the time (in blue)
Water occurrence less than  55 % of the time are in white


In [ ]:
#@markdown Landsat scenes from both Tier 1 and Tier 2 of Collection 2 are 
#@markdown included for possible use. For Tier 2, specify the maximum spatial 
#@markdown error (in meters) for pixels. Only orthorectified images (L1T) are 
#@markdown considered. The default value is 24 meters.
rmse = 24  #@param {type: "number"}
#@markdown __Run this block__ after inputting the above parameter
print(f"Tier 2 L1T scenes with a maximum root mean square error of {rmse} meters will be considered")

Tier 2 L1T scenes with a maximum root mean square error of 24 meters will be considered


In [ ]:
#@markdown Landsat thermal band records begin in 1982 with Landsat 4 Thematic Mapper (TM). 
#@markdown Enter the year range that you want to search, as well as months of 
#@markdown the year. To search all months, `m1 = 1` and `m2 = 12` <p>
#@markdown <p>First year (y1) and last year (y2). Years are inclusive, minimum value is 1982. <p>
#@markdown <p>First month (m1) and last month (m2). Months are inclusive.<p>
#@markdown <p>Note that this method has only been tested for temperature estimation where there is no ice present on the lake. 

y1 = 1982 #@param {type: "number"}
y2 =  2020#@param {type: "number"}

m1 =  5#@param {type: "number"}
m2 = 11 #@param {type: "number"}
print('Script will search years %d through %d and months %d through %d of each year' % (y1,y2,m1,m2))
#@markdown __Run this block__ after inputting the above parameters

Script will search years 1982 through 2020 and months 5 through 11 of each year


# **2. Find water pixels & delineate boundary**
This step uses the bounding box or the table file as well as the water persistence using [JRC Water dataset](https://global-surface-water.appspot.com/) to define the waterbody area. 

Note, in some cases the _Opt A_ bounding box method will include upstream and downstream water areas, particularily where there are rivers that are large enough to be detected by Landsat. To avoid this, use the _Opt B_ table file method. <p>
You can explore the JRC Water dataset [here](https://global-surface-water.appspot.com/).<p>

In [ ]:
#@markdown This code block reduces the bounding box defined above to the largest water body present, and then counts the 30m pixels within the water body.<p>
#@markdown Landsat scenes included in analysis must have a minimum percentage of clear lake pixels available (these are discontinuous pixels). Default is 25%.
pctLakeCoverage = 25 #@param {type: "number"}
#@markdown __Run this block__ after inputting the above parameter

if use_user_input_file=="yes":
  try:
    box = shp
  except NameError:
    raise NameError('''Cannot find user-input file. Are you using Option B or 
    Opt A bounding box? You may need to correct and rerun.''')

wateroccurrence = sw.select(0)
water = wateroccurrence.gte(pctTime)
water = water.updateMask(water.neq(0))
 

#########################
# Find the biggest waterbody in the bounding box
def addArea(feature):
  # returns area +/- 1sqm
  return feature.set({'area':feature.geometry().area(1)})  

regions = water.addBands(wateroccurrence).reduceToVectors(
    reducer=ee.Reducer.min(),
    geometry=box,
    scale=30,
    maxPixels=5e9,
    geometryInNativeProjection=False,
    labelProperty='surfaceWater').map(addArea).sort('area',False);

lake_outline = ee.Feature(regions.first())
geo = lake_outline.geometry()

try:
  #########################
  # Find the best utm zone over the lake to generate CRS

  utm = utmbounds.filterBounds(box)
  utmJoin = ee.Join.saveBest('matches','best');
  utmCondition = ee.Filter.intersects(**{
      "rightField": '.geo',
      "leftField": '.geo'
  })
  utmJoined = utmJoin.apply(lake_outline, utm, utmCondition)
  
  try:
    getZone = utmJoined.getInfo()["features"][0]["properties"]["matches"]["properties"]["ZONE"]
    getHemi = utmJoined.getInfo()["features"][0]["properties"]["matches"]["properties"]["HEMISPHERE"]

    #indicate hemisphere  
    if getHemi == 'n':
      crs_out = f"EPSG:326{getZone}"
    else:
      crs_out = f"EPSG:327{getZone}"
    print("Data will be calculated with output CRS:",crs_out)
    #########################
    # print the user some pixel count statistics

    watercount = water.setDefaultProjection(crs_out).reduceRegion(reducer=ee.Reducer.count().unweighted(), 
                                    geometry=lake_outline.geometry(),
                                    scale=30, bestEffort=True)
    lakesurface = watercount.getInfo()["occurrence"]  # total pixels over lake

    pixel_min = round(lakesurface*(pctLakeCoverage/100.0))
    print("\nTotal # of lake pixels: ", lakesurface);
    print(f"{pctLakeCoverage}% of available lake pixels is", pixel_min)

    if lakesurface < 42:
      print('''Because lakeCoSTR only supplies aggregated data, 
      the lake you have selected has too few pixels available for 
      meaningful data aggregation. 
      It is not recommended that you use lakeCoSTR for this lake.''')
    else:
      #########################
      # Find the overlapping landsat path/rows

      pathrow = wrs2.filterBounds(geo)
        
      num_of_pr = len(pathrow.getInfo()["features"])
      path_west = 1
      path_east = 233
      row_north = 122
      row_south = 1

      for i in range(0,num_of_pr):
        pr = pathrow.getInfo()["features"][i]["properties"]["PR"]
        p = pr[:3]
        r = pr[3:]
        if int(p) > path_west:
          path_west = int(p)
        if int(p) < path_east:
          path_east = int(p)
        if int(r) < row_north:
          row_north = int(r)
        if int(r) > row_south:
          row_south = int(r)

      p1 = path_west
      p2 =  path_east
      r1 =  row_north
      r2 =  row_south
      print(f"\nYour lake overlaps {num_of_pr} landsat footprint(s)")
      print(f'bounding path(s): {path_west} (west) to {path_east} (east)')
      print(f'bounding row(s): {row_north} (north) to {row_south} (south)\n')


      ##### Attempt to find the "best" path/row by comparing footprints that completely
      ## contain the lake, or if none completely contain, find the footprint that has
      ## the most overlap

      footprintJoinAll = ee.Join.saveAll('matches')
      footprintJoinBest = ee.Join.saveBest(**{'matchKey':'matches','measureKey':'best'})

      footprintContains = ee.Filter.isContained(**{'rightField':'.geo','leftField':'.geo'})
      footprintIntersects = ee.Filter.intersects(**{'rightField':'.geo','leftField':'.geo'})

      allContained = footprintJoinAll.apply(lake_outline,pathrow,footprintContains)
      anyIntersects = footprintJoinBest.apply(lake_outline,pathrow,footprintIntersects)

      try:
        containedMatches = allContained.getInfo()["features"][0]["properties"]["matches"]
      except IndexError:
        containedMatches = []

      intersectedMatches = anyIntersects.getInfo()["features"][0]["properties"]["matches"]

      #########################
      # Display a map

      pixel_fig = folium.Figure(width="50%")
      pixel_map = folium.Map(location=[center_lat,center_lon],zoom_start=6).add_to(pixel_fig)
      basemaps['Google Maps'].add_to(pixel_map)
      pixel_map.add_ee_layer(pathrow,{},'path row')

      def addFPtoMap(list_of_fps):
        for fp in list_of_fps:
          makeGeojson = folium.GeoJson(fp["geometry"])
          fpPath = str(fp["properties"]["PATH"])
          fpRow = str(fp["properties"]["ROW"])
          popup = folium.Popup("P"+fpPath+" R"+fpRow)
          popup.add_to(makeGeojson)
          makeGeojson.add_to(pixel_map)

      if len(containedMatches)>0: #lake is fully contained by at least 1 path-row
        if num_of_pr == 1: 
          addFPtoMap(containedMatches)
          print('''Great news! Your lake is completely contained by a single path-row footprint.
          Because your lake does not cross multiple WRS2 rows, there's nothing more you need to do.
          You can skip the next codeblock and move to the 'Get Skin Surface Temperatures' section.
          ''')
        elif len(intersectedMatches) == 0: #and the lake doesn't have any partial matches
          addFPtoMap(containedMatches)
          if(row_north-row_south) == 0: #if it is contained by only one row
            print('''Great news! Your lake is completely contained by at least one path-row footprint.
            Because your lake does not cross multiple WRS2 rows, there's nothing more you need to do.
            You can skip the next codeblock and move to the 'Get Skin Surface Temperatures' section.
            ''')
          else: #if it is contained by multiple rows
            print('''Great news! Your lake is completely contained by multiple WRS2 path-row footprints, 
            HOWEVER, those footprints include multiple WRS2 rows in the same WRS2 path. In order to not retreive near-duplicate 
            temperature estimates from neghboring WRS2 rows, please *manually input* a single ROW value for `new_row_north` and 
            `new_row_south` in the codeblock below. We reccommend you choose the row that is closest to the equator (a lower row number in the Northern 
            Hemisphere, or a higher row number in the Southern Hemisphere).
            ''')
            print("Click the blue highlighted footprint to see the path and row")
        elif len(intersectedMatches)>0: #if it has partial matches
          if len(containedMatches) >0: #and it has fully contained matches
            addFPtoMap(containedMatches)
            print('''Great news! Your lake is completely contained by at least one path-row footprint, 
            HOWEVER, you need to *manually input* the path and row values for the blue box(es) displayed 
            in the map below into the next code block because the overlapping path-row combinations 
            outlined in black do/does not completely contain your lake.
            ''')
            print("Click the blue highlighted footprint to see the path and row")
      else: #otherwise, you're outta luck
        print('''
        CAUTION: Your lake is not completely contained by a single WRS2 path-row footprint. lakeCoSTR functions best
        when your lake is completely contained by at least one WRS2 path-row footprint. Because of this, we do not 
        recommend the use of lakeCoSTR to acquire surface temperature estimates for your lake. 
        ''')

      pixel_map.add_ee_layer(lake_outline.geometry(), {}, 'lake area')
      display(pixel_map)
  except:
    print('''An error occurred - either you have different selections of lake area and 
    water occurrance area (double check sections 1.3 and 1.4) or the lake you selected is 
    too large to process in lakeCoSTR.''')
except IndexError:
  print('''An error occurred - this could be because there are no water pixels available for analysis.
  Check the map in above step to see if there are water pixels available for analysis.''')

Data will be calculated with output CRS: EPSG:32618

Total # of lake pixels:  17165
25% of available lake pixels is 4291

Your lake overlaps 1 landsat footprint(s)
bounding path(s): 13 (west) to 13 (east)
bounding row(s): 30 (north) to 30 (south)

Great news! Your lake is completely contained by a single path-row footprint.
          Because your lake does not cross multiple WRS2 rows, there's nothing more you need to do.
          You can skip the next codeblock and move to the 'Get Skin Surface Temperatures' section.
          


In [ ]:
#@markdown If ther previous code block directed you to `*manually input*` WRS2 path-row values, 
#@markdown please enter the values for path ('P') and row ('R') as indicated by clicking on the
#@markdown blue box in the map above. 

#@markdown <br>If you weren't directed to change your the WRS2 path-row parameters, leave these blank.

new_path_west = "" #@param {type:'string'}
new_path_east = "" #@param {type: 'string'}
new_row_north = "" #@param {type: 'string'}
new_row_south = "" #@param {type: 'string'}

if new_path_west:
  p1 = int(new_path_west)
else: 
  p1 =  path_west
if new_path_east:
  p2 =  int(new_path_east)
else:
  p2 = path_east
if new_row_north:
  r1 =  int(new_row_north)
else:
  r1 = row_north
if new_row_south:
  r2 =  int(new_row_south)
else:
  r2 = row_south

print(f'new bounding path(s): {p1} (west) to {p2} (east)')
print(f'new bounding row(s): {r1} (north) to {r2} (south)\n')




# **3. Get Skin Surface Temperatures**

In [ ]:

#@markdown **Run this block** to filter by date and site location, convert 
#@markdown temperature from K --> C, compile and stack images, and carry over the metadata. 

#For landsat 8, any scenes with possible image quality 
#issues due to Scene Select Mirror (SSM) position are removed.

l4 = (l4t1.merge(l4t2).filterMetadata('L1_PROCESSING_LEVEL','equals','L1TP') \
      .filterMetadata('GEOMETRIC_RMSE_MODEL','not_greater_than',rmse) \
      .filterBounds(geo) \
      .filter(ee.Filter.calendarRange(y1,y2,'year')) \
      .filter(ee.Filter.calendarRange(m1,m2,'month')) \
      .filterMetadata('WRS_ROW','not_less_than',r1).filterMetadata('WRS_ROW','not_greater_than',r2) \
      .filterMetadata('WRS_PATH','not_greater_than',p1).filterMetadata('WRS_PATH','not_less_than',p2) \
      .map(prep457bands, True))

l5 = (l5t1.merge(l5t2).filterMetadata('L1_PROCESSING_LEVEL','equals','L1TP') \
      .filterMetadata('GEOMETRIC_RMSE_MODEL','not_greater_than',rmse) \
      .filterBounds(geo) \
      .filter(ee.Filter.calendarRange(y1,y2,'year')) \
      .filter(ee.Filter.calendarRange(m1,m2,'month')) \
      .filterMetadata('WRS_ROW','not_less_than',r1).filterMetadata('WRS_ROW','not_greater_than',r2) \
      .filterMetadata('WRS_PATH','not_greater_than',p1).filterMetadata('WRS_PATH','not_less_than',p2) \
      .map(prep457bands, True))

l7 = (l7t1.merge(l7t2).filterMetadata('L1_PROCESSING_LEVEL','equals','L1TP') \
      .filterMetadata('GEOMETRIC_RMSE_MODEL','not_greater_than',rmse) \
      .filterBounds(geo) \
      .filter(ee.Filter.calendarRange(y1,y2,'year')) \
      .filter(ee.Filter.calendarRange(m1,m2,'month')) \
      .filterMetadata('WRS_ROW','not_less_than',r1).filterMetadata('WRS_ROW','not_greater_than',r2) \
      .filterMetadata('WRS_PATH','not_greater_than',p1).filterMetadata('WRS_PATH','not_less_than',p2) \
      .map(prep457bands, True))

l8 = (l8t1.merge(l8t2).filterMetadata('L1_PROCESSING_LEVEL','equals','L1TP') \
      .filterMetadata('GEOMETRIC_RMSE_MODEL','not_greater_than',rmse) \
      .filterBounds(geo) \
      .filter(ee.Filter.calendarRange(y1,y2,'year')) \
      .filter(ee.Filter.calendarRange(m1,m2,'month')) \
      .filterMetadata('WRS_ROW','not_less_than',r1).filterMetadata('WRS_ROW','not_greater_than',r2) \
      .filterMetadata('WRS_PATH','not_greater_than',p1).filterMetadata('WRS_PATH','not_less_than',p2) \
      .filterMetadata('TIRS_SSM_POSITION_STATUS','not_equals','SWITCHED') \
      .map(prep8bands, True))

landsat = ee.ImageCollection(l4).merge(l5).merge(l7).merge(l8)

print("Landsat compiled at", strftime("%x %X"))

'''
Add metadata to each scene that indicates how many visible pixels were included 
in analysis, and filter images so that only scenes with the minimum number of 
pixels remain.
'''
def countPixels(img):
  img = ee.Image(img)
  getCount = img.reduceRegion(**{
      "reducer": ee.Reducer.count(),
      "geometry": geo, 
      "scale": 30})
  count = ee.Dictionary(getCount).get('surface_temp')
  return img.set('pixel_count',count)
before = landsat.map(countPixels)
countedPixels = before.filterMetadata('pixel_count','not_less_than', pixel_min)
print('Total number of LS scenes:',len(before.getInfo()['features']),'\nNumber of LS scenes after filter:', len(countedPixels.getInfo()['features']))
print("Landsat filtered:", strftime("%x %X"))

Landsat compiled at 09/06/22 21:08:46
Total number of LS scenes: 603 
Number of LS scenes after filter: 319
Landsat filtered: 09/06/22 21:09:23


# **4. Data Visualization**
The code blocks in the following section allow you to explore your results using dataframes and pixel-value distributions. You can visually inspect each scene for errant data or export the data to explore further.

In [ ]:
#@title 4.1 Convert data from server-side EE objects to client-side Python objects
#@markdown __Run this block__ to define functions for data exploration


# These functions convert image pixels to numpy arrays
def createArrays(img):
  psi_prop = ee.Image(img).sampleRectangle(region=geo, defaultValue=-1)
  getProp = psi_prop.get("surface_temp")
  return img.set("img_array", getProp)

def getArrays(client_img_col):
  list_of_arrays = []
  for img in client_img_col:
    prop = img["properties"]["img_array"]
    a = np.array(prop)
    list_of_arrays.append(a)
  return list_of_arrays
print('Conversion function imported.')

def generate_histogram(img):
  img = ee.Image(img)
  fhisto = img.reduceRegion(**{
      "reducer": ee.Reducer.fixedHistogram(-5,30,70).unweighted(),
      "geometry": geo,
      "scale": 30,
      "maxPixels": 5e9
  })
  return img.set('histogram',fhisto.get('surface_temp'))
print("Will generate histograms with a fixed x-axis between -5 - 30 deg C in 0.5 degree bins")


Conversion function imported.
Will generate histograms with a fixed x-axis between -5 - 30 deg C in 0.5 degree bins


In [ ]:
# @markdown This code block executes the functions from the previous block and 
# @markdown converts the image collection from a server-side EE object to a client-side 
# @markdown Python object. This allows iteration over the image collection with a `for` 
# @markdown loop, which provides much more Python functionality.<p><mark>__This step 
# @markdown could take some time to run</mark> depending on the length of the 
# @markdown ImageCollection.__

# @markdown If you recieve an error at this step that `your response size exceeds limit`, 
# @markdown you will need to go back and reduce the number of years of data you are requesting. 

print("Start:", strftime("%x %X"))
imgs_histo = countedPixels.map(generate_histogram)
generate_arrays = imgs_histo.map(createArrays)

imgCol = generate_arrays.getInfo()["features"]
print("Finish:", strftime("%x %X"))
print("Number of Landsat scenes: ", len(imgCol))

Start: 09/06/22 21:10:25
Finish: 09/06/22 21:11:43
Number of Landsat scenes:  319


In [ ]:
#@markdown Now that the image collection is client-side, __run this block__ to 
#@markdown get arrays of each image.
# returns a list of arrays
imgs_to_arrays = getArrays(imgCol)
print("Converted images to arrays")

Converted images to arrays


In [ ]:
#@title 4.2 Collated pixel distributions
#@markdown __Run this block__ to iterate through the image collection and convert 
#@markdown all bins and frequencies to a single Pandas `DataFrame`. 

length_plots = len(imgCol)
list_of_dfs = []
list_of_uids = []

# fig = make_subplots(rows=length_plots, cols=1)
for i in imgCol:
  uid = (i['properties']['L1_LANDSAT_PRODUCT_ID'])
  list_of_uids.append(uid)
  histogram = i["properties"].get("histogram")

  a = pd.DataFrame(histogram, columns=["bin",uid]).set_index("bin")
  # fig.add_histogram()
  list_of_dfs.append(a)

appended_dfs = pd.concat(list_of_dfs,ignore_index=False, axis=1)
transposed_dfs = appended_dfs.T

In [ ]:
#@title 4.3 View pixel distribution using `matplotlib`
#@markdown __Run this block__ to generate histograms for each lake temperature 
#@markdown Landsat scene. Each scene is saved as png in a folder 'histograms' in the lakeCoSTR_output directory. 
#@markdown This step can take some time to run.

#print(len(list_of_uids)," histograms will be generated")
print("Start:", strftime("%x %X"))

#make 'histogram' folder in defined directory
if not os.path.exists(os.path.join(output_dir, 'histograms')):
    os.makedirs(os.path.join(output_dir, 'histograms'))

histo_dir = os.path.join(output_dir, 'histograms')

print("Histograms will be saved at ", histo_dir)

histos = appended_dfs
#list of scenes
scenes = histos.columns
histos = pd.DataFrame.transpose(histos) #reorient data
#reset index
histos = histos.reset_index()
histos_vert = histos.melt(id_vars=['index'])

#iterate over the scene list
for i in range(0, len(scenes)):
  df = histos_vert[histos_vert['index'] == scenes[i]]
  histoname = os.path.join(histo_dir,scenes[i]+'_histo.png')
  plt.bar(x = df.bin, height = df.value)
  plt.ylabel('frequency')
  plt.xlabel('temperature (deg C)')
  plt.title(scenes[i])
  plt.savefig(histoname)
  print('Saving histogram for Landsat scene ', scenes[i], ' at ', histoname)  
  plt.close()

print("Finish:", strftime("%x %X"), 'Your histograms should now be visible at ', histo_dir)

Start: 09/06/22 21:11:53
Histograms will be saved at  /content/drive/MyDrive/lakeCoSTR_output/sunapee_v1_12/histograms
Saving histogram for Landsat scene  LT04_L1TP_013030_19870526_20201014_02_T1  at  /content/drive/MyDrive/lakeCoSTR_output/sunapee_v1_12/histograms/LT04_L1TP_013030_19870526_20201014_02_T1_histo.png
Saving histogram for Landsat scene  LT04_L1TP_013030_19890819_20200916_02_T1  at  /content/drive/MyDrive/lakeCoSTR_output/sunapee_v1_12/histograms/LT04_L1TP_013030_19890819_20200916_02_T1_histo.png
Saving histogram for Landsat scene  LT04_L1TP_013030_19900721_20200915_02_T1  at  /content/drive/MyDrive/lakeCoSTR_output/sunapee_v1_12/histograms/LT04_L1TP_013030_19900721_20200915_02_T1_histo.png
Saving histogram for Landsat scene  LT05_L1TP_013030_19840610_20200918_02_T1  at  /content/drive/MyDrive/lakeCoSTR_output/sunapee_v1_12/histograms/LT05_L1TP_013030_19840610_20200918_02_T1_histo.png
Saving histogram for Landsat scene  LT05_L1TP_013030_19840930_20200918_02_T1  at  /conten

# **5. Export Landsat temperature statistics to CSV**

In [ ]:
#@markdown Execute this cell block to export a CSV containing temperature and related image metadata to your Google Drive.

def exportWholeLakeStats(img):
  img = ee.Image(img).clip(geo)

  # retrieve image metadata for output file
  landsattime = img.get('system:time_start')
  cloudcover_pct_scene = img.get('CLOUD_COVER')
  esd_au_scene = img.get("EARTH_SUN_DISTANCE")
  sunelev_deg_scene = img.get('SUN_ELEVATION')
  sunazi_deg_scene = img.get('SUN_AZIMUTH')
  solzenang_deg_scene = img.get('SOLAR_ZENITH_ANGLE')
  count = img.get('pixel_count')
  pctAvail = ee.Number(count).divide(lakesurface).multiply(100)

  # get statistics for skin_temperature
  reducers = ee.Reducer.mean().combine(**{
      "reducer2": ee.Reducer.stdDev(), "sharedInputs":True}).combine(**{
      "reducer2": ee.Reducer.minMax(), "sharedInputs":True}).combine(**{
      "reducer2": ee.Reducer.median(**{"maxBuckets":500, "minBucketWidth": 0.125}), "sharedInputs":True}).combine(**{
      "reducer2": ee.Reducer.skew(), "sharedInputs":True}).combine(**{
      "reducer2": ee.Reducer.kurtosis(), "sharedInputs":True}).combine(**{
      "reducer2": ee.Reducer.percentile(**{"percentiles": [25,75], "maxBuckets": 500, "minBucketWidth": 0.125}), "sharedInputs":True})
      
  
  stats = img.reduceRegion(**{
      "reducer": reducers,
      "geometry": geo,
      "scale": 30,
      "crs": crs_out,
      "maxPixels": 5e9
  })
  
  more_stats = ee.Dictionary({'availablePixels_count':count,
                            'datetime_landsat':landsattime,
                            'cloudcover_pct_scene':cloudcover_pct_scene,
                            'sunelev_deg_scene':sunelev_deg_scene,
                            'sunazi_deg_scene':sunazi_deg_scene,
                            'esd_au_scene':esd_au_scene,
                            'sunazi_deg_scene':sunazi_deg_scene,
                            'lakeCoverage_pct':pctAvail,
                            #'datetime_landsat_excelformat':ee.Number(landsattime).divide(1000.0).divide(86400).add(25569)
                            })
  stats2 = ee.Dictionary.combine(stats, more_stats)

  return ee.Feature(None,stats2)

temp_stats = countedPixels.map(exportWholeLakeStats)

#create the file name with user-specified prefix
temp_filename = (file_prefix+'_temp_stats')

export_task = ee.batch.Export.table.toDrive(**{
    'collection': temp_stats,
    'description': temp_filename,
    "fileFormat": "CSV",
    'folder': short_dir
})

print("Exporting ", temp_filename+".csv")
export_task.start()
print('Polling for task (id: {}) at'.format(export_task.id))

while export_task.active():
  print(strftime("%x %X"), export_task.status())
  sleep(10)

print("Finished:", strftime("%x %X"))

print('Export should now be visible in Drive at path:\n',os.path.join(output_dir, temp_filename + ".csv"))

Exporting  sunapee_v1_12_temp_stats.csv
Polling for task (id: ZUXLNN7ROPFCI37RHPJ7VGJM) at
09/06/22 21:13:39 {'state': 'READY', 'description': 'sunapee_v1_12_temp_stats', 'creation_timestamp_ms': 1662498819420, 'update_timestamp_ms': 1662498819420, 'start_timestamp_ms': 0, 'task_type': 'EXPORT_FEATURES', 'id': 'ZUXLNN7ROPFCI37RHPJ7VGJM', 'name': 'projects/earthengine-legacy/operations/ZUXLNN7ROPFCI37RHPJ7VGJM'}
09/06/22 21:13:50 {'state': 'READY', 'description': 'sunapee_v1_12_temp_stats', 'creation_timestamp_ms': 1662498819420, 'update_timestamp_ms': 1662498819420, 'start_timestamp_ms': 0, 'task_type': 'EXPORT_FEATURES', 'id': 'ZUXLNN7ROPFCI37RHPJ7VGJM', 'name': 'projects/earthengine-legacy/operations/ZUXLNN7ROPFCI37RHPJ7VGJM'}
09/06/22 21:14:01 {'state': 'READY', 'description': 'sunapee_v1_12_temp_stats', 'creation_timestamp_ms': 1662498819420, 'update_timestamp_ms': 1662498819420, 'start_timestamp_ms': 0, 'task_type': 'EXPORT_FEATURES', 'id': 'ZUXLNN7ROPFCI37RHPJ7VGJM', 'name': 'pro

# **6. Pair Landsat with *in situ* data**
This next section is optional, but allows you to compare any field data you may have to the Landsat data produced here. This section uses the CSV exported above.<p>
In order for this to run successfully, your data must be in CSV format and have the following headers/columns:


*  `datetime`: Date and time of each temperature reading in a recognized strftime format
*  `temp_degC`: an integer or float number representing temperature in degrees Celcius
*  `location`: for in-lake statistics, a column with lake zone names (string format, no special characters); can be all the same for a single output or differentiated by lake zones/sensors
* `depth_m`: as an integer or float number, for statistics about the depth of sensors for matched scenes

The file may have additional columns, but the above are mandatory fields. To see an example dataset, click here. 


## 6.1 Upload CSV of *insitu* data
Choose an option below. For more ways to import data, see [here](https://colab.research.google.com/notebooks/io.ipynb)  

In [ ]:
#@title Please provide some information about your data
#@markdown What timezone is your data? <a href="https://en.wikipedia.org/wiki/List_of_tz_database_time_zones" target="_blank">Olson Times Wikipedia Page</a> Specifically, you need to indicate the UTC offset for your data for proper pairing, including whether or not your data timestamp observes Daylight Savings Time. Enter the text for the matching UTC offset and DST behavior from the 'TZ database name' column in the linked table. Note that the sign of the GMT offset is intentionally inverted from the UTC offset.
insitu_timezone = "Etc/GMT+5" #@param {type:"string"}
#@markdown How is your datetime formatted? Please use <a href='https://strftime.org/' target='_blank'>strftime</a> format.
datetime_format = "%Y-%m-%d %H:%M:%S" #@param {type:"string"}

In [ ]:
#@title Option 1: Link to raw CSV from Github
pasted_path = "" #@param {type:"string"}

is_df = pd.read_csv(pasted_path)
#is_df["datetime"] = pd.to_datetime(is_df["datetime"])
print("dataframe created")

In [ ]:
#@title Option 2: Upload CSV from your Google Drive folder
#@markdown using the folder icon on the left hand side, navigate to the file in
#@markdown your Google Drive (named 'drive' here), click on the three vertical dots to the right of the file
#@markdown and click on 'copy file path'. Paste the copied path below.

pasted_path = "" #@param {type:"string"}

is_df = pd.read_csv(pasted_path)
print("dataframe created")

In [ ]:
#@title Option 3: Upload CSV from your local file system to Colab
#@markdown (this is temporary while connected to the current Colab runtime session)<br>
#@markdown Run this cell to upload a file
uploaded = files.upload()
for fn in uploaded.keys():
  print("Paste this in the box below:\n/content/{name}".format(name=fn))


Saving insitu_temp_data_v2021-10-20.csv to insitu_temp_data_v2021-10-20.csv
Paste this in the box below:
/content/insitu_temp_data_v2021-10-20.csv


In [ ]:
pasted_path = "/content/insitu_temp_data_v2021-10-20.csv" #@param {type:"string"}

is_df = pd.read_csv(pasted_path)
print("dataframe created")

dataframe created


## 6.2 Pair *insitu* data with Landsat data

In [ ]:
#@title Convert local datetime to UTC time
#@markdown __Run this block__ to apply conversion of your specified local time to UTC time. 
insitu_tz = pytz.timezone(insitu_timezone)
landsat_tz = pytz.timezone("UTC")

def convert_datetime(dt, dtformat=datetime_format):
  #converts string format insitu time to a datetime obj
  dto = datetime.strptime(dt, datetime_format)
  #makes it time aware
  dto_local = insitu_tz.localize(dto)
  #converts to utc
  dto_utc = dto_local.astimezone(landsat_tz)
  return dto_utc

dtobj_series = is_df['datetime']
dtobj_series_conv = dtobj_series.apply(convert_datetime)
is_df['datetime_utc'] = dtobj_series_conv

print("\nDate/time function imported and datetime converted in dataframe", strftime("%x %X"))


Date/time function imported and datetime converted in dataframe 09/06/22 21:27:39


In [ ]:
#@markdown Enter the window of time (in minutes) from Landsat flyover where *insitu* data should be included. For example, `timewindow = 30` will include any data within 60 minutes of Landsat flyover (+/- 30 minutes)
timewindow = 30 #@param {type:"number"

cwd = output_dir
os.chdir(cwd)

outfile = (file_prefix + "_temp_landsat_paired.csv")

print(f"Time window: +- {timewindow} minutes")
print(f"In-situ time Zone: {insitu_timezone}")
print("\nLandsat input file:\n", os.path.join(cwd,temp_filename+".csv"))
print("\nOutput file will be saved to:\n", os.path.join(cwd,outfile))

min_sec = timewindow * 60


Time window: +- 30 minutes
In-situ time Zone: Etc/GMT+5

Landsat input file:
 /content/drive/MyDrive/lakeCoSTR_output/sunapee_v1_12/sunapee_v1_12_temp_stats.csv

Output file will be saved to:
 /content/drive/MyDrive/lakeCoSTR_output/sunapee_v1_12/sunapee_v1_12_temp_landsat_paired.csv


In [ ]:
#@markdown The following codeblock compares the datasets and generate statistics 
#@markdown for in-lake data that matches Landsat flyovers.<br> 
#@markdown It also uses a subfolder called 'ancillary' (and creates the folder if needed) 
#@markdown that keeps a record of any insitu data points that are used for each Landsat scene.
#@markdown If you have a large number of scenes to pair and/or a large dataframe of
#@markdown in-situ data, this step may take some time. Timestamps that print below are 
#@markdown the Landast scene acquisition times where there are in-situ matches.

print("Start:", strftime("%x %X"))
file_name = (output_dir + '/' + temp_filename + '.csv')
gee_csv = pd.read_csv(file_name)
insitu_csv = is_df

#make 'ancillary' folder in defined directory
if not os.path.exists(os.path.join(cwd, 'ancillary')):
    os.makedirs(os.path.join(cwd, 'ancillary'))
print('Matched records files will be visible at ', cwd+ '/ancillary')

#filter GEE output so that it's only as recent as minimum in-situ data
insitumin = min(insitu_csv.datetime_utc)
insitumax = max(insitu_csv.datetime_utc)

def conv_lstime(i):
  return pd.Timestamp(i, unit = 'ms', tz = 'utc')

gee_csv['landsat_time_utc'] = gee_csv.datetime_landsat.apply(conv_lstime)

gee_overlap = gee_csv[gee_csv.landsat_time_utc>insitumin]
gee_overlap_fin = gee_overlap[gee_overlap.landsat_time_utc<insitumax]
#reindex filtered dataset
gee_overlap_fin = gee_overlap_fin.reset_index(drop=True)

gee_datelist = gee_overlap_fin["landsat_time_utc"]

# function to convert dt_delta to seconds
def delta_tosec(i):
  total_secs = i.seconds + i.days*24*60*60
  return total_secs

for i in range(0,len(gee_datelist)):
  scene = gee_overlap_fin['system:index'][i][-20:]
  landsattime = gee_overlap_fin['landsat_time_utc'][i]

  # create a df of insitu data that are observed within the user-specified cutoff 

  #calculate time delta for each obs
  insitu_csv['dt_delta'] = landsattime - insitu_csv['datetime_utc']
  insitu_csv['delta_secs'] = insitu_csv.dt_delta.apply(delta_tosec)  
  same_time = insitu_csv[(abs(insitu_csv.delta_secs) <= min_sec)]
  
  #if the dataframe is not empty, summarize the results
  if same_time.shape[0]>0:
    print(landsattime)
    gee_overlap_fin.loc[i, "scene"] = scene
    gee_overlap_fin.loc[i, "is_temp_avg"] = same_time["temp_degC"].mean()
    gee_overlap_fin.loc[i, "is_temp_stdev"] = same_time["temp_degC"].std()
    gee_overlap_fin.loc[i, "is_depth_avg"] = same_time["depth_m"].mean()
    gee_overlap_fin.loc[i, "is_depth_stdev"] = same_time["depth_m"].std()
    gee_overlap_fin.loc[i, "is_temp_med"] = same_time["temp_degC"].median()
    gee_overlap_fin.loc[i, "insitu_count"] = same_time.shape[0]

    site_stats = same_time.groupby(['location'])['temp_degC'].agg(['median', 'mean', 'std', 'count'])

    sites = site_stats.axes[0]
    stats = site_stats.axes[1]

    for site in sites:
        for stat in stats:
            newcol = "{0}_{1}".format(str(site), str(stat))
            gee_overlap_fin.loc[i, newcol] = site_stats[stat][site].item()

    if same_time.shape[0] > 0:
        same_time.to_csv(os.path.join(cwd, 'ancillary', scene + ".csv"))

out_csv = gee_overlap_fin[gee_overlap_fin["insitu_count"] > 0]
out_csv.to_csv(os.path.join(cwd, outfile))
("Finished:", strftime("%x %X"))

Start: 09/06/22 21:27:46
Matched records files will be visible at  /content/drive/MyDrive/lakeCoSTR_output/sunapee_v1_12/ancillary
2006-07-25 15:26:04.693000+00:00
2006-09-11 15:26:38.218000+00:00
2007-07-12 15:26:35.703000+00:00
2007-08-13 15:26:16.380000+00:00
2007-08-29 15:26:05.072000+00:00
2007-09-14 15:25:59.297000+00:00
2008-05-11 15:21:30.840000+00:00
2008-06-12 15:20:39.651000+00:00
2008-08-31 15:18:22.672000+00:00
2008-10-18 15:16:53.316000+00:00
2009-05-30 15:20:55.701000+00:00
2009-08-18 15:22:15.552000+00:00
2009-09-19 15:22:42.884000+00:00
2009-10-21 15:23:03.546000+00:00
2009-11-22 15:23:17.660000+00:00
2010-06-02 15:23:35.349000+00:00
2010-06-18 15:23:31.122000+00:00
2010-07-04 15:23:26.405000+00:00
2010-09-06 15:23:08.713000+00:00
2010-09-22 15:23:03.139000+00:00
2010-10-08 15:22:55.004000+00:00
2011-06-21 15:22:11.277000+00:00
2011-07-07 15:22:05.247000+00:00
2011-08-24 15:21:39.563000+00:00
2011-10-11 15:20:56.341000+00:00
2011-11-12 15:20:26.133000+00:00
2006-07-17 

('Finished:', '09/06/22 21:41:02')

##You are all set. Go analyze your lake surface temperature data! 

We suggest you save a copy of this notebook in the same folder as your downloaded data, which will appear in your Google Drive in the 'lakeCoSTR_output' folder for good measure.
